In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
SEED = 2024
random.seed(SEED)
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score, precision_score
from imblearn.metrics import specificity_score, sensitivity_score
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from sklearn.utils import check_array, _safe_indexing, sparsefuncs_fast, check_X_y, check_random_state
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse
from sklearn.neighbors import NearestNeighbors
from sklearn.base import clone
from numbers import Integral
from sklearn.svm import SVC
from collections import Counter
import seaborn as sns
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, SMOTENC, SVMSMOTE
import os
# import missingpy as missingpy
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
import pickle
#from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import LabelEncoder
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, precision_recall_curve, f1_score, roc_auc_score, make_scorer
from sklearn.model_selection import cross_val_score, cross_validate

In [2]:
data = pd.read_excel('Loan_default.xlsx')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51940 entries, 0 to 51939
Columns: 680 entries, id to loss
dtypes: float64(517), int64(162), uint64(1)
memory usage: 269.5 MB


In [4]:
data

,id,loan_status,f1,f2,f3,f4,f5,f6,f7,f8,...,f769,f771,f772,f773,f774,f775,nominal_3,nominal_4,f778,loss
0,3,0,126,10,0.500080,1100,3,83607,1800,1527,...,-5.18,2.89,-1.73,1.04,0.2521,0.7258,1,0,5,0
1,5,0,109,9,0.502749,2900,4,79124,89,491,...,-11.12,6.11,-3.82,2.51,0.2282,-0.5399,0,0,5,0
2,7,0,121,9,0.985674,2900,4,13026,4565,263,...,-11.42,7.06,-4.99,3.77,0.2458,-0.6732,0,0,5,0
3,8,1,128,9,0.385778,2900,4,79244,6597,3592,...,-7.11,4.45,-3.26,2.56,0.2947,-0.7220,0,0,5,1
4,9,0,126,9,0.745471,2900,4,78920,3058,112,...,-3.29,2.02,-1.35,0.95,0.2601,0.7132,0,0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51935,105464,1,136,8,0.420418,2200,4,3793,7820,11541,...,-11.81,8.26,-6.08,4.60,0.2199,0.9908,0,0,93,3
51936,105465,0,127,8,0.839374,2200,4,7458,7548,3440,...,-5.34,3.53,-2.54,1.90,0.2605,0.7329,0,0,93,0
51937,105466,0,140,8,0.888521,2200,4,9836,6377,4286,...,-1.80,1.42,-1.17,0.98,0.3367,0.9228,0,0,93,0
51938,105467,0,126,8,0.006620,2200,4,76831,331,2366,...,-1.38,1.07,-0.92,0.82,0.3809,-0.5489,0,0,93,0


In [5]:
data = data.sample(n=6000, random_state=2025).reset_index(drop=True)

In [6]:
data.isnull().sum()

id             0
loan_status    0
f1             0
f2             0
f3             0
              ..
f775           0
nominal_3      0
nominal_4      0
f778           0
loss           0
Length: 680, dtype: int64

In [7]:
data.head()

,id,loan_status,f1,f2,f3,f4,f5,f6,f7,f8,...,f769,f771,f772,f773,f774,f775,nominal_3,nominal_4,f778,loss
0,63501,0,163,7,0.239081,5100,10,82134,1850,341,...,-3.75,2.40,-1.73,1.34,0.2833,-0.2578,0,1,67,0
1,14905,0,126,10,0.582750,1300,3,510,4223,1463,...,-4.61,2.88,-2.02,1.56,0.2747,0.2589,1,0,10,0
2,10599,0,122,8,0.934541,2300,4,78252,3841,2813,...,-2.59,1.83,-1.38,1.07,0.1956,0.4659,1,0,13,0
3,47782,0,118,10,0.472430,1300,16,86027,3764,5484,...,-4.87,2.51,-1.47,0.87,0.2109,-0.5439,1,0,393,0
4,51763,0,153,8,0.129895,5600,4,81227,64,1265,...,-18.78,12.15,-8.72,6.56,0.2660,0.5302,0,1,10,0


In [8]:
data.drop('id', axis=1, inplace =True)

In [9]:
from collections import Counter
data.rename(columns = {'loan_status': 'target'},inplace = True)
print(Counter(data.target))

Counter({0: 5420, 1: 580})


In [10]:
data.dropna(inplace=True)

In [11]:
encoded_df = data.drop('target',axis = 1)
df_target= data[['target']]

In [12]:
print(Counter(data.target))

Counter({0: 5420, 1: 580})


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(encoded_df, df_target, test_size=0.3, random_state=SEED)

In [14]:
from sklearn.preprocessing import StandardScaler
print(Counter(y_train['target']))
sc = StandardScaler().set_output(transform="pandas")
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

Counter({0: 3796, 1: 404})


#### smote-enn-enc

In [15]:
class MySMOTEENNENC:
    
    def __init__(self, categorical_features):
        self.categorical_features = categorical_features

    def chk_neighbors(self, nn_object, additional_neighbor):
        if isinstance(nn_object, Integral):
            return NearestNeighbors(n_neighbors=nn_object + additional_neighbor)
        elif isinstance(nn_object, KNeighborsMixin):
            return clone(nn_object)
        else:
            raise ValueError("nn_object should be either an integer or KNeighborsMixin instance")
    
    def generate_samples(self, X, nn_data, nn_num, rows, cols, steps, continuous_features_):
        rng = check_random_state(42)
        diffs = nn_data[nn_num[rows, cols]] - X[rows]

        if sparse.issparse(X):
            steps = sparse.csr_matrix(steps)
            X_new = X[rows] + steps.multiply(diffs)
        else:
            X_new = X[rows] + steps * diffs

        X_new = X_new.tolil() if sparse.issparse(X_new) else X_new
        nn_data = nn_data.toarray() if sparse.issparse(nn_data) else nn_data

        all_neighbors = nn_data[nn_num[rows]]

        for idx in range(continuous_features_.size, X.shape[1]):
            mode = stats.mode(all_neighbors[:, :, idx], axis=1)[0]
            X_new[:, idx] = np.ravel(mode)
        
        return X_new
    
    def make_samples(self, X, y_dtype, y_type, nn_data, nn_num, n_samples, continuous_features_, step_size=1.0):
        random_state = check_random_state(42)
        samples_indices = random_state.randint(low=0, high=len(nn_num.flatten()), size=n_samples)
        steps = step_size * random_state.uniform(size=n_samples)[:, np.newaxis]
        rows = np.floor_divide(samples_indices, nn_num.shape[1])
        cols = np.mod(samples_indices, nn_num.shape[1])

        X_new = self.generate_samples(X, nn_data, nn_num, rows, cols, steps, continuous_features_)
        y_new = np.full(n_samples, fill_value=y_type, dtype=y_dtype)
        
        return X_new, y_new
    
    def cat_corr_pandas(self, X, target_df, target_column, target_value):
        categorical_columns = list(X.columns)
        X = pd.concat([X, target_df], axis=1)
        is_target = X.loc[:, target_column] == target_value
        X_filtered = X.loc[is_target, :]
        X_filtered.drop(target_column, axis=1, inplace=True)
        nrows = len(X)
        encoded_dict_list = []
        nan_dict = dict()
        c = 0
        imb_ratio = len(X_filtered) / len(X)
        
        for column in categorical_columns:
            encoded_dict = {}
            for level in list(X.loc[:, column].unique()):
                row_level_filter = X.loc[:, column] == level
                rows_in_level = len(X.loc[row_level_filter, :])
                O = len(X.loc[is_target & row_level_filter, :])
                E = rows_in_level * imb_ratio
                
                if E == 0:  # Prevent division by zero
                    print(f"Category '{level}' in column '{column}' has zero expected count (E = {E}). Skipping encoding.")
                    ENC = 0  # Default value if E == 0
                else:
                    ENC = (O - E) / E  # The encoding formula
                
                encoded_dict[level] = ENC

            encoded_dict_list.append(encoded_dict)
            X.loc[:, column] = X[column].map(encoded_dict)
            nan_idx_array = np.ravel(np.argwhere(np.isnan(X.loc[:, column])))
            
            if len(nan_idx_array) > 0:
                nan_dict[c] = nan_idx_array
            c += 1
            X.loc[:, column].fillna(-1, inplace=True)
        
        X.drop(target_column, axis=1, inplace=True)
        return X, encoded_dict_list, nan_dict

    def fit_resample(self, X, y):
        X_cat_encoded, encoded_dict_list, nan_dict = self.cat_corr_pandas(X.iloc[:, np.asarray(self.categorical_features)], y, target_column='target', target_value=1)
        X_cat_encoded = np.array(X_cat_encoded)
        y = np.ravel(y)
        X = np.array(X)

        unique, counts = np.unique(y, return_counts=True)
        target_stats = dict(zip(unique, counts))
        n_sample_majority = max(target_stats.values())
        class_majority = max(target_stats, key=target_stats.get)
        sampling_strategy = {key: n_sample_majority - value for (key, value) in target_stats.items() if key != class_majority}

        n_features_ = X.shape[1]
        categorical_features = np.asarray(self.categorical_features)
        if categorical_features.dtype.name == 'bool':
            categorical_features_ = np.flatnonzero(categorical_features)
        else:
            if any([cat not in np.arange(n_features_) for cat in categorical_features]):
                raise ValueError('Some of the categorical indices are out of range. Indices should be between 0 and {}'.format(n_features_ - 1))
            categorical_features_ = categorical_features

        continuous_features_ = np.setdiff1d(np.arange(n_features_), categorical_features_)

        target_stats = Counter(y)
        class_minority = min(target_stats, key=target_stats.get)

        X_continuous = X[:, continuous_features_]
        X_continuous = check_array(X_continuous, accept_sparse=['csr', 'csc'])
        X_minority = _safe_indexing(X_continuous, np.flatnonzero(y == class_minority))

        if sparse.issparse(X):
            if X.format == 'csr':
                _, var = sparse.csr_matrix(X_minority).mean(axis=0, keepdims=True), sparse.csr_matrix(X_minority).var(axis=0, keepdims=True)
            else:
                _, var = sparse.csc_matrix(X_minority).mean(axis=0, keepdims=True), sparse.csc_matrix(X_minority).var(axis=0, keepdims=True)
        else:
            var = X_minority.var(axis=0)
        median_std_ = np.median(np.sqrt(var))

        X_categorical = X[:, categorical_features_]
        X_copy = np.hstack((X_continuous, X_categorical))
        X_cat_encoded = X_cat_encoded * median_std_
        X_encoded = np.hstack((X_continuous, X_cat_encoded))

        X_resampled = X_encoded.copy()
        y_resampled = y.copy()

        for class_sample, n_samples in sampling_strategy.items():
            if n_samples == 0:
                continue
            target_class_indices = np.flatnonzero(y == class_sample)
            X_class = _safe_indexing(X_encoded, target_class_indices)
            nn_k_ = self.chk_neighbors(5, 1)
            nn_k_.fit(X_class)

            nns = nn_k_.kneighbors(X_class, return_distance=False)[:, 1:]
            X_new, y_new = self.make_samples(X_class, y.dtype, class_sample, X_class, nns, n_samples, continuous_features_, 1.0)

            if sparse.issparse(X_new):
                X_resampled = sparse.vstack([X_resampled, X_new])
                sparse_func = 'tocsc' if X.format == 'csc' else 'tocsr'
                X_resampled = getattr(X_resampled, sparse_func)()
            else:
                X_resampled = np.vstack((X_resampled, X_new))
            y_resampled = np.hstack((y_resampled, y_new))

        X_resampled_copy = X_resampled.copy()
        i = 0
        for col in range(continuous_features_.size, X.shape[1]):
            encoded_dict = encoded_dict_list[i]
            i += 1
            for key, value in encoded_dict.items():
                X_resampled_copy[:, col] = np.where(np.round(X_resampled_copy[:, col], 4) == np.round(value * median_std_, 4), key, X_resampled_copy[:, col])

        for key, value in nan_dict.items():
            for item in value:
                X_resampled_copy[item, continuous_features_.size + key] = X_copy[item, continuous_features_.size + key]

        X_resampled = X_resampled_copy
        indices_reordered = np.argsort(np.hstack((continuous_features_, categorical_features_)))
        if sparse.issparse(X_resampled):
            col_indices = X_resampled.indices.copy()
            for idx, col_idx in enumerate(indices_reordered):
                mask = X_resampled.indices == col_idx
                col_indices[mask] = idx
            X_resampled.indices = col_indices
        else:
            X_resampled = X_resampled[:, indices_reordered]

        return self.enn(X_resampled, y_resampled)
    
    def enn(self, X, y):
        nn = KNeighborsClassifier(n_neighbors=3)
        nn.fit(X, y)
        y_pred = nn.predict(X)

        mask = np.array([y[i] == y_pred[i] for i in range(len(y))])
        X_clean = X[mask]
        y_clean = y[mask]

        return X_clean, y_clean


##### geometric-smote-nc

In [16]:
from sklearn.neighbors import NearestNeighbors
from sklearn.base import clone
from sklearn.utils import check_random_state
from collections import Counter
from scipy import stats
import pandas as pd
from scipy import sparse
from sklearn.utils.validation import check_array
import numpy as np

class GeometricSMOTENC:
    
    def __init__(self, categorical_features):
        self.categorical_features = categorical_features
        
    def chk_neighbors(self, nn_object, additional_neighbor):
        if isinstance(nn_object, int):
            return NearestNeighbors(n_neighbors=nn_object + additional_neighbor)
        elif isinstance(nn_object, KNeighborsMixin):
            return clone(nn_object)
        else:
            raise ValueError(f"Unsupported nn_object type: {type(nn_object)}")

    def generate_samples(self, X, nn_data, nn_num, rows, cols, steps, continuous_features_):
        rng = check_random_state(42)

        # Convert X and nn_data to NumPy arrays if they're DataFrames
        if isinstance(X, pd.DataFrame):
            X = X.to_numpy()
        if isinstance(nn_data, pd.DataFrame):
            nn_data = nn_data.to_numpy()
        
        diffs = nn_data[nn_num[rows, cols]] - X[rows]
        
        # Geometric scaling of the difference
        gamma = rng.uniform(low=0, high=1, size=steps.shape[0])
        steps_geometric = np.power(diffs, gamma[:, np.newaxis])

        if sparse.issparse(X):
            sparse_func = type(X).__name__
            steps_geometric = getattr(sparse, sparse_func)(steps_geometric)
            X_new = X[rows] * steps_geometric
        else:
            X_new = X[rows] * steps_geometric

        # Handle categorical features - use mode of nearest neighbors
        nn_data = nn_data.toarray() if sparse.issparse(nn_data) else nn_data
        all_neighbors = nn_data[nn_num[rows]]

        # For categorical features, retain the most frequent category (mode) among the neighbors
        for idx in continuous_features_:
            mode = stats.mode(all_neighbors[:, :, idx], axis=1)[0]
            X_new[:, idx] = np.ravel(mode)

        return X_new
    
    def make_samples(self, X, y_dtype, y_type, nn_data, nn_num, n_samples, continuous_features_, step_size=1.0):
        random_state = check_random_state(42)
        samples_indices = random_state.randint(low=0, high=len(nn_num.flatten()), size=n_samples)    
        steps = step_size * random_state.uniform(size=n_samples)[:, np.newaxis]
        rows = np.floor_divide(samples_indices, nn_num.shape[1])
        cols = np.mod(samples_indices, nn_num.shape[1])

        X_new = self.generate_samples(X, nn_data, nn_num, rows, cols, steps, continuous_features_)
        y_new = np.full(n_samples, fill_value=y_type, dtype=y_dtype)
        
        return X_new, y_new

    def fit_resample(self, X, y):
        categorical_features = np.asarray(self.categorical_features)
        n_features_ = X.shape[1]

        if any([cat not in np.arange(n_features_) for cat in categorical_features]):
            raise ValueError(f"Some of the categorical indices are out of range. Indices should be between 0 and {n_features_}")

        categorical_features_ = np.flatnonzero(categorical_features)
        continuous_features_ = np.setdiff1d(np.arange(n_features_), categorical_features_)

        unique, counts = np.unique(y, return_counts=True)
        target_stats = dict(zip(unique, counts))
        n_sample_majority = max(target_stats.values())
        class_majority = max(target_stats, key=target_stats.get)
        sampling_strategy = {key: n_sample_majority - value for (key, value) in target_stats.items() if key != class_majority}

        if isinstance(X, pd.DataFrame):  # Convert X to NumPy array
            X = X.to_numpy()

        y = np.ravel(y)
        minority_class = min(target_stats, key=target_stats.get)

        X_resampled = X.copy()
        y_resampled = y.copy()

        for class_sample, n_samples in sampling_strategy.items():
            if n_samples == 0:
                continue
            target_class_indices = np.flatnonzero(y == class_sample)
            X_class = X[target_class_indices]
            nn_k_ = self.chk_neighbors(5, 1)
            nn_k_.fit(X_class)

            nns = nn_k_.kneighbors(X_class, return_distance=False)[:, 1:]
            X_new, y_new = self.make_samples(X, y.dtype, class_sample, X_class, nns, n_samples, continuous_features_, 1.0)

            X_resampled = np.vstack((X_resampled, X_new))
            y_resampled = np.hstack((y_resampled, y_new))

        return X_resampled, y_resampled



### smote-enc

In [17]:
class MySMOTEENC:
    def __init__(self, categorical_features):
        self.categorical_features = categorical_features

    def chk_neighbors(self, nn_object, additional_neighbor):
        if isinstance(nn_object, Integral):
            return NearestNeighbors(n_neighbors=nn_object + additional_neighbor)
        elif isinstance(nn_object, KNeighborsMixin):
            return clone(nn_object)
        else:
            raise TypeError("Invalid neighbor object type.")     

    def generate_samples(self, X, nn_data, nn_num, rows, cols, steps, continuous_features_):
        rng = check_random_state(42)

        diffs = nn_data[nn_num[rows, cols]] - X[rows]

        if sparse.issparse(X):
            sparse_func = type(X).__name__
            steps = getattr(sparse, sparse_func)(steps)
            X_new = X[rows] + steps.multiply(diffs)
        else:
            X_new = X[rows] + steps * diffs 

        X_new = (X_new.tolil() if sparse.issparse(X_new) else X_new)
        # convert to dense array since scipy.sparse doesn't handle 3D
        nn_data = (nn_data.toarray() if sparse.issparse(nn_data) else nn_data)

        all_neighbors = nn_data[nn_num[rows]]

        for idx in range(continuous_features_.size, X.shape[1]):
            mode = stats.mode(all_neighbors[:, :, idx], axis=1)[0]
            X_new[:, idx] = np.ravel(mode)
            
        return X_new

    def make_samples(self, X, y_dtype, y_type, nn_data, nn_num, n_samples, continuous_features_, step_size=1.0):
        random_state = check_random_state(42)
        samples_indices = random_state.randint(low=0, high=len(nn_num.flatten()), size=n_samples)    
        steps = step_size * random_state.uniform(size=n_samples)[:, np.newaxis]
        rows = np.floor_divide(samples_indices, nn_num.shape[1])
        cols = np.mod(samples_indices, nn_num.shape[1])

        X_new = self.generate_samples(X, nn_data, nn_num, rows, cols, steps, continuous_features_)
        y_new = np.full(n_samples, fill_value=y_type, dtype=y_dtype)
        
        return X_new, y_new

    def cat_corr_pandas(self, X, target_df, target_column, target_value):
        # X has categorical columns
        categorical_columns = list(X.columns)
        X = pd.concat([X, target_df], axis=1)

        # filter X for target value
        is_target = X.loc[:, target_column] == target_value
        X_filtered = X.loc[is_target, :]

        X_filtered.drop(target_column, axis=1, inplace=True)

        # get columns in X
        nrows = len(X)
        encoded_dict_list = []
        nan_dict = dict({})
        c = 0
        imb_ratio = len(X_filtered) / len(X) if len(X) > 0 else 0 
        OE_dict = {}
        
        for column in categorical_columns:
            for level in list(X.loc[:, column].unique()):
                
                # filter rows where level is present
                row_level_filter = X.loc[:, column] == level
                rows_in_level = len(X.loc[row_level_filter, :])
                
                # number of rows in level where target is 1
                O = len(X.loc[is_target & row_level_filter, :])
                E = rows_in_level * imb_ratio
                # Encoded value = chi, i.e. (observed - expected)/expected
                if E == 0:
                    ENC = 0
                else:
                    ENC = (O - E) / E

                OE_dict[level] = ENC
                
            encoded_dict_list.append(OE_dict)

            X.loc[:, column] = X[column].map(OE_dict)
            nan_idx_array = np.ravel(np.argwhere(np.isnan(X.loc[:, column])))
            
            if len(nan_idx_array) > 0:
                nan_dict[c] = nan_idx_array
            c = c + 1
            X.loc[:, column].fillna(-1, inplace=True)
            
        X.drop(target_column, axis=1, inplace=True)
        return X, encoded_dict_list, nan_dict

    def fit_resample(self, X, y):
        X_cat_encoded, encoded_dict_list, nan_dict = self.cat_corr_pandas(X.iloc[:, np.asarray(self.categorical_features)], y, target_column='target', target_value=1)
        X_cat_encoded = np.array(X_cat_encoded)
        y = np.ravel(y)
        X = np.array(X)

        unique, counts = np.unique(y, return_counts=True)
        target_stats = dict(zip(unique, counts))
        n_sample_majority = max(target_stats.values())
        class_majority = max(target_stats, key=target_stats.get)
        sampling_strategy = {key: n_sample_majority - value for (key, value) in target_stats.items() if key != class_majority}

        n_features_ = X.shape[1]
        categorical_features = np.asarray(self.categorical_features)
        if categorical_features.dtype.name == 'bool':
            categorical_features_ = np.flatnonzero(categorical_features)
        else:
            if any([cat not in np.arange(n_features_) for cat in categorical_features]):
                raise ValueError('Some of the categorical indices are out of range. Indices'
                            ' should be between 0 and {}'.format(n_features_))
            categorical_features_ = categorical_features

        continuous_features_ = np.setdiff1d(np.arange(n_features_), categorical_features_)

        target_stats = Counter(y)
        class_minority = min(target_stats, key=target_stats.get)

        X_continuous = X[:, continuous_features_]
        X_continuous = check_array(X_continuous, accept_sparse=['csr', 'csc'])
        X_minority = _safe_indexing(X_continuous, np.flatnonzero(y == class_minority))

        if sparse.issparse(X):
            if X.format == 'csr':
                _, var = sparsefuncs_fast.csr_mean_variance_axis0(X_minority)
            else:
                _, var = sparsefuncs_fast.csc_mean_variance_axis0(X_minority)
        else:
            var = X_minority.var(axis=0)
        median_std_ = np.median(np.sqrt(var))

        X_categorical = X[:, categorical_features_]
        X_copy = np.hstack((X_continuous, X_categorical))

        X_cat_encoded = X_cat_encoded * median_std_

        X_encoded = np.hstack((X_continuous, X_cat_encoded))
        X_resampled = X_encoded.copy()
        y_resampled = y.copy()

        for class_sample, n_samples in sampling_strategy.items():
            if n_samples == 0:
                continue
            target_class_indices = np.flatnonzero(y == class_sample)
            X_class = _safe_indexing(X_encoded, target_class_indices)
            nn_k_ = self.chk_neighbors(5, 1)
            nn_k_.fit(X_class)

            nns = nn_k_.kneighbors(X_class, return_distance=False)[:, 1:]
            X_new, y_new = self.make_samples(X_class, y.dtype, class_sample, X_class, nns, n_samples, continuous_features_, 1.0)

            if sparse.issparse(X_new):
                X_resampled = sparse.vstack([X_resampled, X_new])
                sparse_func = 'tocsc' if X.format == 'csc' else 'tocsr'
                X_resampled = getattr(X_resampled, sparse_func)()
            else:
                X_resampled = np.vstack((X_resampled, X_new))
            y_resampled = np.hstack((y_resampled, y_new))
            
        X_resampled_copy = X_resampled.copy()
        i = 0
        for col in range(continuous_features_.size, X.shape[1]):
            encoded_dict = encoded_dict_list[i]
            i = i + 1
            for key, value in encoded_dict.items():
                X_resampled_copy[:, col] = np.where(np.round(X_resampled_copy[:, col], 4) == np.round(value * median_std_, 4), key, X_resampled_copy[:, col])

        for key, value in nan_dict.items():
            for item in value:
                X_resampled_copy[item, continuous_features_.size + key] = X_copy[item, continuous_features_.size + key]

               
        X_resampled = X_resampled_copy   
        indices_reordered = np.argsort(np.hstack((continuous_features_, categorical_features_)))
        if sparse.issparse(X_resampled):
            col_indices = X_resampled.indices.copy()
            for idx, col_idx in enumerate(indices_reordered):
                mask = X_resampled.indices == col_idx
                col_indices[mask] = idx
            X_resampled.indices = col_indices
        else:
            X_resampled = X_resampled[:, indices_reordered]
        return X_resampled, y_resampled


### smote-nc

In [18]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from collections import Counter
from sklearn.utils import check_random_state
from joblib import Parallel, delayed

class SMOTENC:
    def __init__(self, categorical_features, k_neighbors=5, random_state=42, n_jobs=-1):
        self.categorical_features = np.asarray(categorical_features)
        self.k_neighbors = k_neighbors
        self.random_state = random_state
        self.n_jobs = n_jobs

    def _generate_synthetic_sample(self, X_cont, X_cat, neighbors, rng, idx):
        """Generate a single synthetic sample"""
        neighbor_idx = rng.choice(neighbors[idx])
        alpha = rng.uniform()
        
        # Interpolate continuous features
        cont_new = X_cont[idx] + alpha * (X_cont[neighbor_idx] - X_cont[idx])
        
        # Randomly select categorical features
        cat_new = np.where(rng.random(len(X_cat[idx])) < 0.5,
                         X_cat[idx], 
                         X_cat[neighbor_idx])
        
        return np.hstack((cont_new, cat_new))

    def fit_resample(self, X, y):
        """Optimized implementation of SMOTE-NC resampling"""
        rng = check_random_state(self.random_state)
        
        # Convert to numpy arrays if DataFrames
        if isinstance(X, pd.DataFrame):
            X = X.values
        if isinstance(y, (pd.Series, pd.DataFrame)):
            y = y.values
        
        # Convert y to 1D array of hashable types (strings or numbers)
        y = np.asarray(y).flatten()
        
        # Identify minority and majority classes
        try:
            class_counts = Counter(y)
        except TypeError:
            # If elements are unhashable (like arrays), convert to tuples
            y = [tuple(row) if isinstance(row, np.ndarray) else row for row in y]
            class_counts = Counter(y)
            
        if len(class_counts) < 2:
            return X, y
            
        minority_class = min(class_counts, key=class_counts.get)
        majority_class = max(class_counts, key=class_counts.get)
        
        # Calculate number of synthetic samples needed
        n_minority = class_counts[minority_class]
        n_majority = class_counts[majority_class]
        n_synthetic = n_majority - n_minority
        
        if n_synthetic <= 0:
            return X, y

        # Get minority samples
        minority_mask = (y == minority_class) if not isinstance(minority_class, tuple) else \
                       [tuple(row) == minority_class if isinstance(row, np.ndarray) else row == minority_class for row in y]
        X_minority = X[minority_mask]
        
        # Split continuous and categorical features
        continuous_features = np.setdiff1d(np.arange(X.shape[1]), self.categorical_features)
        X_cont = X_minority[:, continuous_features]
        X_cat = X_minority[:, self.categorical_features]
        
        # Find nearest neighbors
        nn = NearestNeighbors(n_neighbors=self.k_neighbors+1, n_jobs=self.n_jobs)
        nn.fit(X_cont)
        neighbors = nn.kneighbors(X_cont, return_distance=False)[:, 1:]  # Exclude self
        
        # Generate synthetic samples in parallel
        synthetic_samples = Parallel(n_jobs=self.n_jobs)(
            delayed(self._generate_synthetic_sample)(
                X_cont, X_cat, neighbors, rng, 
                rng.randint(0, len(X_minority))
            ) for _ in range(n_synthetic)
        )
        
        # Combine results
        synthetic_samples = np.array(synthetic_samples)
        
        # Reorder features to original order
        feature_order = np.argsort(np.hstack((continuous_features, self.categorical_features)))
        synthetic_samples = synthetic_samples[:, feature_order]
        
        X_resampled = np.vstack((X, synthetic_samples))
        y_resampled = np.hstack((y, np.full(n_synthetic, minority_class)))
        
        return X_resampled, y_resampled

In [19]:
categorical_features = [0,3,4,5,6,8,9,10,11,12]

In [20]:
##SMOTE NC 

smote_nc = SMOTENC(categorical_features=categorical_features, 
                   k_neighbors=3, 
                   random_state=42,
                   n_jobs=-1)  # Use all available cores

# Apply SMOTENC
X_S_NC, y_S_NC = smote_nc.fit_resample(X_train_sc, y_train)

In [21]:
## LETS APPLY SMOTE_enn-ENC TO OUR DATA
from sklearn.neighbors import KNeighborsClassifier
samp_pipeline = MySMOTEENNENC(categorical_features = categorical_features)
X_ENN_ENC, y_ENN_ENC = samp_pipeline.fit_resample(X_train_sc, y_train)

In [22]:
col_list = list(X_train_sc.columns)
X_ENN_ENC = pd.DataFrame(X_ENN_ENC, index=range(X_ENN_ENC.shape[0]),
                          columns=col_list)
y_ENN_ENC= pd.DataFrame(y_ENN_ENC)
y_ENN_ENC['target'] = y_ENN_ENC[0]
y_ENN_ENC.drop(0,axis=1, inplace=True)

In [23]:
# Initialize and run the Geometric SMOTENC
g_smote = GeometricSMOTENC(categorical_features=categorical_features)
X_g_smote, y_g_smote = g_smote.fit_resample(X_train_sc, y_train)

In [24]:
## LETS APPLY SMOTE_ENC TO OUR DATA
samp_pipeline = MySMOTEENC(categorical_features = categorical_features )
X_ENC, y_ENC = samp_pipeline.fit_resample(X_train_sc, y_train)

In [25]:
col_list = list(X_train_sc.columns)
X_ENC = pd.DataFrame(X_ENC, index=range(X_ENC.shape[0]),
                          columns=col_list)
y_ENC= pd.DataFrame(y_ENC)
y_ENC['target'] = y_ENC[0]
y_ENC.drop(0,axis=1, inplace=True)

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import (
    balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score,
    confusion_matrix, precision_recall_curve, auc
)
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from tqdm import tqdm

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score,
    confusion_matrix, precision_recall_curve, auc
)
from sklearn.utils import shuffle, resample
from sklearn.impute import SimpleImputer
from tqdm import tqdm

# Define Adaptive Lasso functions
alpha = 0.000011
gprime = lambda w: 1. / (2. * np.sqrt(np.abs(w)) + np.finfo(float).eps)

# Initialize the imputer to fill missing values with the mean
imputer = SimpleImputer(strategy='mean')

# Apply imputer to the training and test data
X_ENN_ENC_imputed = imputer.fit_transform(X_ENN_ENC)
X_test_sc_imputed = imputer.transform(X_test_sc)  # Make sure this is defined

# Prepare results storage
results = []

# Train and evaluate Adaptive Lasso 100 times using Random Forest
for i in tqdm(range(1, 101), desc="Training Adaptive Lasso with Random Forest"):
    # Randomly sample 80% of the data (introduces variation)
    X_train_subset, y_train_subset = resample(
        X_ENN_ENC_imputed, y_ENN_ENC, n_samples=int(0.8 * len(X_ENN_ENC_imputed)), random_state=i
    )
    
    # Shuffle data
    X_train_shuffled, y_train_shuffled = shuffle(X_train_subset, y_train_subset, random_state=i)

    # Convert to NumPy array
    X_train_array = np.asarray(X_train_shuffled)

    # Initialize weights randomly (0.1 to 10.0)
    np.random.seed(i)
    weights = np.random.uniform(0.1, 10.0, size=X_train_array.shape[1])

    # Randomize alpha slightly
    current_alpha = alpha * np.random.uniform(0.8, 1.2)

    # Iterative process for Adaptive Lasso
    for _ in range(5):
        X_w = X_train_array / weights
        X_w = X_w + np.random.normal(0, 1e-2, X_w.shape)  # Larger jitter

        # Fit Random Forest model
        try:
            model = RandomForestClassifier(
                n_estimators=250,
                max_depth=None,
                min_samples_split=2,
                min_samples_leaf=1,
                random_state=i,
                class_weight='balanced'
            )
            model.fit(X_w, y_train_shuffled)
        except Exception as e:
            print(f"Error in iteration {i}: {e}")
            continue

        # Update weights based on feature importances
        new_weights = np.asarray(gprime(model.feature_importances_))
        if new_weights.shape[0] != X_train_array.shape[1]:
            new_weights = np.pad(new_weights, (0, X_train_array.shape[1] - new_weights.shape[0]), 'constant', constant_values=1)
        weights = new_weights.reshape(-1)

    # Predict with random threshold (0.4 to 0.6)
    try:
        y_proba = model.predict_proba(X_test_sc_imputed)[:, 1]
        threshold = np.random.uniform(0.4, 0.6)
        y_pred = (y_proba >= threshold).astype(int)

        # Compute metrics
        balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_proba)

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        specificity = tn / (tn + fp)
        g_mean = np.sqrt(recall * specificity)

        precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_proba)
        pr_auc = auc(recall_vals, precision_vals)

        # Store results
        results.append([
            i, balanced_accuracy, precision, recall, f1, roc_auc, g_mean, pr_auc, threshold
        ])
    except Exception as e:
        print(f"Error in iteration {i}: {e}")
        continue

# Convert to DataFrame
df_results = pd.DataFrame(
    results,
    columns=['iteration', 'balanced_accuracy', 'precision', 'recall', 'f1_score', 'roc_auc', 'g_mean', 'pr_auc', 'threshold']
)

# Save results
df_results.to_csv('LOANDEFAULT_RF_ALASSO_RF(SMOTE_ENNENC).csv', index=False)

# Display
print(df_results.head())

Training Adaptive Lasso with Random Forest: 100%|█| 100/100 [3:50:38<00:00, 138.

   iteration  balanced_accuracy  precision    recall  f1_score   roc_auc  \
0          1           0.856597   0.658416  0.755682  0.703704  0.940346   
1          2           0.818406   0.245614  0.954545  0.390698  0.950340   
2          3           0.915053   0.806452  0.852273  0.828729  0.972287   
3          4           0.891961   0.574713  0.852273  0.686499  0.955787   
4          5           0.844436   0.945736  0.693182  0.800000  0.976438   

     g_mean    pr_auc  threshold  
0  0.850632  0.805229   0.474606  
1  0.807003  0.843270   0.418363  
2  0.912896  0.913388   0.441372  
3  0.891078  0.869187   0.462660  
4  0.830779  0.897761   0.489590  


In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score,
    confusion_matrix, precision_recall_curve, auc
)
from sklearn.utils import shuffle, resample
from sklearn.impute import SimpleImputer
from tqdm import tqdm

# Define Adaptive Lasso functions
alpha = 0.000011
gprime = lambda w: 1. / (2. * np.sqrt(np.abs(w)) + np.finfo(float).eps)

# Initialize the imputer to fill missing values with the mean
imputer = SimpleImputer(strategy='mean')

# Apply imputer to the training and test data
X_ENC_imputed = imputer.fit_transform(X_ENC)
X_test_sc_imputed = imputer.transform(X_test_sc)  # Make sure this is defined

# Prepare results storage
results = []

# Train and evaluate Adaptive Lasso 100 times using Random Forest
for i in tqdm(range(1, 101), desc="Training Adaptive Lasso with Random Forest"):
    # Randomly sample 80% of the data (introduces variation)
    X_train_subset, y_train_subset = resample(
        X_ENC_imputed, y_ENC, n_samples=int(0.8 * len(X_ENC_imputed)), random_state=i
    )
    
    # Shuffle data
    X_train_shuffled, y_train_shuffled = shuffle(X_train_subset, y_train_subset, random_state=i)

    # Convert to NumPy array
    X_train_array = np.asarray(X_train_shuffled)

    # Initialize weights randomly (0.1 to 10.0)
    np.random.seed(i)
    weights = np.random.uniform(0.1, 10.0, size=X_train_array.shape[1])

    # Randomize alpha slightly
    current_alpha = alpha * np.random.uniform(0.8, 1.2)

    # Iterative process for Adaptive Lasso
    for _ in range(5):
        X_w = X_train_array / weights
        X_w = X_w + np.random.normal(0, 1e-2, X_w.shape)  # Larger jitter

        # Fit model with variable regularization
        try:
            model = RandomForestClassifier(
                n_estimators=250,
                random_state=i,
                class_weight='balanced'  # Added to handle class imbalance
            )
            model.fit(X_w, y_train_shuffled)
        except Exception as e:
            print(f"Error in iteration {i}: {e}")
            continue

        # Update weights using feature importances instead of coefficients
        new_weights = np.asarray(gprime(model.feature_importances_))
        if new_weights.shape[0] != X_train_array.shape[1]:
            new_weights = np.pad(new_weights, (0, X_train_array.shape[1] - new_weights.shape[0]), 'constant', constant_values=1)
        weights = new_weights.reshape(-1)

    # Predict with random threshold (0.4 to 0.6)
    try:
        y_proba = model.predict_proba(X_test_sc_imputed)[:, 1]
        threshold = np.random.uniform(0.4, 0.6)
        y_pred = (y_proba >= threshold).astype(int)

        # Compute metrics
        balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_proba)

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        specificity = tn / (tn + fp)
        g_mean = np.sqrt(recall * specificity)

        precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_proba)
        pr_auc = auc(recall_vals, precision_vals)

        # Store results
        results.append([
            i, balanced_accuracy, precision, recall, f1, roc_auc, g_mean, pr_auc, threshold
        ])
    except Exception as e:
        print(f"Error in iteration {i}: {e}")
        continue

# Convert to DataFrame
df_results = pd.DataFrame(
    results,
    columns=['iteration', 'balanced_accuracy', 'precision', 'recall', 'f1_score', 'roc_auc', 'g_mean', 'pr_auc', 'threshold']
)

# Save results
df_results.to_csv('LOANDEFAULT_RF_ALASSO(SMOTE_ENC).csv', index=False)

# Display
print(df_results.head())

Training Adaptive Lasso with Random Forest: 100%|█| 100/100 [4:34:21<00:00, 164.

   iteration  balanced_accuracy  precision    recall  f1_score   roc_auc  \
0          1           0.994318   1.000000  0.988636  0.994286  1.000000   
1          2           0.919643   0.402746  1.000000  0.574225  0.999508   
2          3           0.871921   0.297297  1.000000  0.458333  0.999904   
3          4           0.968750   1.000000  0.937500  0.967742  0.999993   
4          5           0.994010   0.994286  0.988636  0.991453  0.999979   

     g_mean    pr_auc  threshold  
0  0.994302  1.000000   0.532542  
1  0.916125  0.997115   0.435739  
2  0.862463  0.999232   0.418162  
3  0.968246  0.999935   0.546331  
4  0.993996  0.999809   0.502438  


In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score,
    confusion_matrix, precision_recall_curve, auc
)
from sklearn.utils import shuffle, resample
from sklearn.impute import SimpleImputer
from tqdm import tqdm

# Define Adaptive Lasso functions
alpha = 0.000011
gprime = lambda w: 1. / (2. * np.sqrt(np.abs(w)) + np.finfo(float).eps)

# Initialize the imputer to fill missing values with the mean
imputer = SimpleImputer(strategy='mean')

# Apply imputer to the training and test data
X_S_NC_imputed = imputer.fit_transform(X_S_NC)
X_test_sc_imputed = imputer.transform(X_test_sc)

# Prepare results storage
results = []

# Train and evaluate Adaptive Lasso 100 times using Random Forest
for i in tqdm(range(1, 101), desc="Training Adaptive Lasso with Random Forest"):
    # Randomly sample 80% of the data
    X_train_subset, y_train_subset = resample(
        X_S_NC_imputed, y_S_NC, n_samples=int(0.8 * len(X_S_NC_imputed)), random_state=i
    )
    
    # Shuffle data
    X_train_shuffled, y_train_shuffled = shuffle(X_train_subset, y_train_subset, random_state=i)

    # Initialize weights
    weights = np.ones(X_train_shuffled.shape[1])
    
    # Iterative process for Adaptive Lasso
    for _ in range(5):
        # Apply weights to features
        X_weighted = X_train_shuffled / weights
        
        # Add small noise for stability
        X_weighted = X_weighted + np.random.normal(0, 1e-3, X_weighted.shape)
        
        # Fit Random Forest
        try:
            model = RandomForestClassifier(
                n_estimators=250,
                max_depth=None,
                min_samples_split=2,
                min_samples_leaf=1,
                random_state=i,
                class_weight='balanced'
            )
            model.fit(X_weighted, y_train_shuffled)
            
            # Update weights using feature importances
            weights = gprime(model.feature_importances_)
            
        except Exception as e:
            print(f"Error in iteration {i}: {e}")
            continue

    # Make predictions
    try:
        # Apply final weights to test data
        X_test_weighted = X_test_sc_imputed / weights
        
        y_proba = model.predict_proba(X_test_weighted)[:, 1]
        threshold = np.random.uniform(0.4, 0.6)
        y_pred = (y_proba >= threshold).astype(int)

        # Calculate metrics
        metrics = {
            'iteration': i,
            'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred),
            'roc_auc': roc_auc_score(y_test, y_proba),
            'threshold': threshold
        }
        
        # Calculate additional metrics
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        metrics.update({
            'specificity': tn / (tn + fp),
            'g_mean': np.sqrt(metrics['recall'] * (tn / (tn + fp))),
            'pr_auc': auc(*precision_recall_curve(y_test, y_proba)[1::-1])
        })
        
        results.append(metrics)
        
    except Exception as e:
        print(f"Prediction error in iteration {i}: {e}")
        continue

# Convert results to DataFrame
df_results = pd.DataFrame(results)

# Save and display results
df_results.to_csv('LOANDEFAULT_RF_ADAPTIVE_LASSO(SMOTE_NC).csv', index=False)
print(df_results.head())

# Feature importance analysis
feature_importances = pd.DataFrame({
    'feature': range(X_S_NC_imputed.shape[1]),
    'importance': weights
}).sort_values('importance', ascending=False)

print("\nTop 10 most important features:")
print(feature_importances.head(10))

Training Adaptive Lasso with Random Forest: 100%|█| 100/100 [4:59:53<00:00, 179.

   iteration  balanced_accuracy  precision    recall  f1_score   roc_auc  \
0          1           0.724432        1.0  0.448864  0.619608  1.000000   
1          2           0.974432        1.0  0.948864  0.973761  0.999986   
2          3           0.968750        1.0  0.937500  0.967742  1.000000   
3          4           0.707386        1.0  0.414773  0.586345  0.999972   
4          5           0.826705        1.0  0.653409  0.790378  1.000000   

   threshold  specificity    g_mean    pr_auc  
0   0.588066          1.0  0.669973  1.000000  
1   0.465870          1.0  0.974096  0.999873  
2   0.471719          1.0  0.968246  1.000000  
3   0.557373          1.0  0.644029  0.999753  
4   0.521814          1.0  0.808337  1.000000  

Top 10 most important features:
     feature  importance
451      451   36.749497
169      169   34.865670
626      626   34.188342
588      588   33.407436
167      167   32.867982
590      590   32.654660
392      392   32.097142
114      114   32.0882

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score,
    confusion_matrix, precision_recall_curve, auc
)
from sklearn.utils import shuffle, resample
from sklearn.impute import SimpleImputer
from tqdm import tqdm

# Define Adaptive Lasso functions
alpha = 0.000011
gprime = lambda w: 1. / (2. * np.sqrt(np.abs(w)) + np.finfo(float).eps)

# Initialize the imputer to fill missing values with the mean
imputer = SimpleImputer(strategy='mean')

# Apply imputer to the training and test data
X_g_smote_imputed = imputer.fit_transform(X_g_smote)
X_test_sc_imputed = imputer.transform(X_test_sc)

# Prepare results storage
results = []

# Train and evaluate Adaptive Lasso 100 times using Random Forest
for i in tqdm(range(1, 101), desc="Training Adaptive Lasso with Random Forest"):
    # Randomly sample 80% of the data (introduces variation)
    X_train_subset, y_train_subset = resample(
        X_g_smote_imputed, y_g_smote, 
        n_samples=int(0.8 * len(X_g_smote_imputed)), 
        random_state=i
    )
    
    # Shuffle data
    X_train_shuffled, y_train_shuffled = shuffle(X_train_subset, y_train_subset, random_state=i)

    # Initialize weights
    weights = np.ones(X_train_shuffled.shape[1])
    
    # Iterative process for Adaptive Lasso
    for _ in range(5):
        # Apply weights to features
        X_weighted = X_train_shuffled / weights
        
        # Add small noise for stability
        X_weighted = X_weighted + np.random.normal(0, 1e-3, X_weighted.shape)
        
        # Fit Random Forest
        try:
            model = RandomForestClassifier(
                n_estimators=250,
                max_depth=None,
                min_samples_split=2,
                min_samples_leaf=1,
                random_state=i,
                class_weight='balanced'
            )
            model.fit(X_weighted, y_train_shuffled)
            
            # Update weights using feature importances
            weights = gprime(model.feature_importances_)
            
        except Exception as e:
            print(f"Error in iteration {i}: {e}")
            continue

    # Make predictions
    try:
        # Apply final weights to test data
        X_test_weighted = X_test_sc_imputed / weights
        
        y_proba = model.predict_proba(X_test_weighted)[:, 1]
        threshold = np.random.uniform(0.4, 0.6)
        y_pred = (y_proba >= threshold).astype(int)

        # Calculate metrics
        metrics = {
            'iteration': i,
            'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred),
            'roc_auc': roc_auc_score(y_test, y_proba),
            'threshold': threshold
        }
        
        # Calculate additional metrics
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        metrics.update({
            'specificity': tn / (tn + fp),
            'g_mean': np.sqrt(metrics['recall'] * (tn / (tn + fp))),
            'pr_auc': auc(*precision_recall_curve(y_test, y_proba)[1::-1])
        })
        
        results.append(metrics)
        
    except Exception as e:
        print(f"Prediction error in iteration {i}: {e}")
        continue

# Convert results to DataFrame
df_results = pd.DataFrame(results)

# Save and display results
df_results.to_csv('LOANDEFAULT_RF_ADAPTIVE_LASSO(GSMOTE_NC).csv', index=False)
print(df_results.head())

# Feature importance analysis
feature_importances = pd.DataFrame({
    'feature': range(X_g_smote_imputed.shape[1]),
    'importance': weights
}).sort_values('importance', ascending=False)

print("\nTop 10 most important features:")
print(feature_importances.head(10))

Training Adaptive Lasso with Random Forest:  47%|▍| 47/100 [2:25:20<2:40:37, 181

#### ADAPTIVE ELASTIC NET

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score,
    confusion_matrix, precision_recall_curve, auc
)
from sklearn.utils import shuffle, resample
from sklearn.impute import SimpleImputer
from tqdm import tqdm

# Define Adaptive Elastic Net parameters with randomization
base_alpha = 0.1  # Base regularization strength
gprime = lambda w: 1. / (2. * np.sqrt(np.abs(w)) + np.finfo(float).eps)

# Initialize the imputer
imputer = SimpleImputer(strategy='mean')

# Apply imputer to data
X_ENN_ENC_imputed = imputer.fit_transform(X_ENN_ENC)
X_test_sc_imputed = imputer.transform(X_test_sc)

# Prepare results storage
results = []

for i in tqdm(range(1, 101), desc="Training Adaptive ElasticNet with RF"):
    # Random subsampling (80% of data)
    X_train_subset, y_train_subset = resample(
        X_ENN_ENC_imputed, y_ENN_ENC, 
        n_samples=int(0.8 * len(X_ENN_ENC_imputed)), 
        random_state=i
    )
    
    # Shuffle data
    X_train_shuffled, y_train_shuffled = shuffle(X_train_subset, y_train_subset, random_state=i)
    X_train_array = np.asarray(X_train_shuffled)

    # Initialize weights
    weights = np.ones(X_train_array.shape[1])
    
    # Randomized hyperparameters
    current_alpha = base_alpha * np.random.uniform(0.7, 1.3)

    # Adaptive Elastic Net iterations
    for _ in range(5):
        X_w = X_train_array / weights
        X_w = X_w + np.random.normal(0, 1e-3, X_w.shape)  # Small jitter
        
        try:
            model = RandomForestClassifier(
                n_estimators=250,
                max_depth=None,
                min_samples_split=2,
                min_samples_leaf=1,
                random_state=i,
                class_weight='balanced'
            )
            model.fit(X_w, y_train_shuffled)
            
            # Update weights using feature importances
            weights = gprime(model.feature_importances_)
            
        except Exception as e:
            print(f"Error in iteration {i}: {e}")
            continue

    # Prediction with random threshold
    try:
        # Apply final weights to test data
        X_test_weighted = X_test_sc_imputed / weights
        
        y_proba = model.predict_proba(X_test_weighted)[:, 1]
        threshold = np.random.uniform(0.4, 0.6)
        y_pred = (y_proba >= threshold).astype(int)

        # Compute metrics
        metrics = {
            'iteration': i,
            'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred),
            'roc_auc': roc_auc_score(y_test, y_proba),
            'threshold': threshold,
            'alpha': current_alpha
        }
        
        # Additional metrics
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        metrics.update({
            'specificity': tn / (tn + fp),
            'g_mean': np.sqrt(metrics['recall'] * (tn / (tn + fp))),
            'pr_auc': auc(*precision_recall_curve(y_test, y_proba)[1::-1])
        })
        
        results.append(metrics)
        
    except Exception as e:
        print(f"Prediction error in iteration {i}: {e}")
        continue

# Convert to DataFrame
df_results = pd.DataFrame(results)

# Save results
df_results.to_csv('LOANDEFAULT_RF_Adaptive_ElasticNet(SMOTE_ENN_ENC).csv', index=False)

# Display results and feature importances
print(df_results.head())

# Feature importance analysis
feature_importances = pd.DataFrame({
    'feature': range(X_ENN_ENC_imputed.shape[1]),
    'importance': weights
}).sort_values('importance', ascending=False)

print("\nTop 10 most important features:")
print(feature_importances.head(10))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score,
    confusion_matrix, precision_recall_curve, auc
)
from sklearn.utils import shuffle, resample
from sklearn.impute import SimpleImputer
from tqdm import tqdm

# Define adaptive weighting function
gprime = lambda w: 1. / (2. * np.sqrt(np.abs(w)) + np.finfo(float).eps)

# Initialize the imputer
imputer = SimpleImputer(strategy='mean')

# Apply imputer to data
X_ENC_imputed = imputer.fit_transform(X_ENC)
X_test_sc_imputed = imputer.transform(X_test_sc)

# Prepare results storage
results = []

for i in tqdm(range(1, 101), desc="Training Adaptive Random Forest"):
    # Random subsampling (80% of data)
    X_train_subset, y_train_subset = resample(
        X_ENC_imputed, y_ENC, 
        n_samples=int(0.8 * len(X_ENC_imputed)), 
        random_state=i
    )
    
    # Shuffle data
    X_train_shuffled, y_train_shuffled = shuffle(X_train_subset, y_train_subset, random_state=i)
    X_train_array = np.asarray(X_train_shuffled)

    # Initialize weights
    weights = np.ones(X_train_array.shape[1])
    
    # Adaptive weighting iterations
    for _ in range(5):
        # Apply weights to features
        X_weighted = X_train_array / weights
        X_weighted = X_weighted + np.random.normal(0, 1e-3, X_weighted.shape)  # Small jitter
        
        try:
            model = RandomForestClassifier(
                n_estimators=250,
                max_depth=None,
                min_samples_split=2,
                min_samples_leaf=1,
                random_state=i,
                class_weight='balanced'
            )
            model.fit(X_weighted, y_train_shuffled)
            
            # Update weights using feature importances
            weights = gprime(model.feature_importances_)
            
        except Exception as e:
            print(f"Training error in iteration {i}: {e}")
            continue

    # Make predictions
    try:
        # Apply final weights to test data
        X_test_weighted = X_test_sc_imputed / weights
        
        y_proba = model.predict_proba(X_test_weighted)[:, 1]
        threshold = np.random.uniform(0.4, 0.6)
        y_pred = (y_proba >= threshold).astype(int)

        # Compute metrics
        metrics = {
            'iteration': i,
            'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred),
            'roc_auc': roc_auc_score(y_test, y_proba),
            'threshold': threshold
        }
        
        # Additional metrics
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        metrics.update({
            'specificity': tn / (tn + fp),
            'g_mean': np.sqrt(metrics['recall'] * (tn / (tn + fp))),
            'pr_auc': auc(*precision_recall_curve(y_test, y_proba)[1::-1])
        })
        
        results.append(metrics)
        
    except Exception as e:
        print(f"Prediction error in iteration {i}: {e}")
        continue

# Convert to DataFrame
df_results = pd.DataFrame(results)

# Save results
df_results.to_csv('LOANDEFAULT_RF_Adaptive_Weighting(SMOTE_ENC).csv', index=False)

# Display results and feature importances
print(df_results.head())

# Feature importance analysis
feature_importances = pd.DataFrame({
    'feature': range(X_ENC_imputed.shape[1]),
    'importance': weights
}).sort_values('importance', ascending=False)

print("\nTop 10 most important features:")
print(feature_importances.head(10))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score,
    confusion_matrix, precision_recall_curve, auc
)
from sklearn.utils import shuffle, resample
from sklearn.impute import SimpleImputer
from tqdm import tqdm

# Define adaptive weighting function
gprime = lambda w: 1. / (2. * np.sqrt(np.abs(w)) + np.finfo(float).eps)

# Initialize the imputer
imputer = SimpleImputer(strategy='mean')

# Apply imputer to data
X_g_smote_imputed = imputer.fit_transform(X_g_smote)
X_test_sc_imputed = imputer.transform(X_test_sc)

# Prepare results storage
results = []

for i in tqdm(range(1, 101), desc="Training Adaptive Random Forest"):
    # Random subsampling (80% of data)
    X_train_subset, y_train_subset = resample(
        X_g_smote_imputed, y_g_smote,
        n_samples=int(0.8 * len(X_g_smote_imputed)),
        random_state=i
    )
    
    # Shuffle data
    X_train_shuffled, y_train_shuffled = shuffle(X_train_subset, y_train_subset, random_state=i)
    X_train_array = np.asarray(X_train_shuffled)

    # Initialize weights
    weights = np.ones(X_train_array.shape[1])
    
    # Adaptive weighting iterations
    for _ in range(5):
        # Apply weights to features
        X_weighted = X_train_array / weights
        X_weighted = X_weighted + np.random.normal(0, 1e-3, X_weighted.shape)  # Small jitter
        
        try:
            model = RandomForestClassifier(
                n_estimators=250,
                max_depth=None,
                min_samples_split=2,
                min_samples_leaf=1,
                random_state=i,
                class_weight='balanced'
            )
            model.fit(X_weighted, y_train_shuffled)
            
            # Update weights using feature importances
            weights = gprime(model.feature_importances_)
            
        except Exception as e:
            print(f"Training error in iteration {i}: {e}")
            continue

    # Make predictions
    try:
        # Apply final weights to test data
        X_test_weighted = X_test_sc_imputed / weights
        
        y_proba = model.predict_proba(X_test_weighted)[:, 1]
        threshold = np.random.uniform(0.4, 0.6)
        y_pred = (y_proba >= threshold).astype(int)

        # Compute metrics
        metrics = {
            'iteration': i,
            'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred),
            'roc_auc': roc_auc_score(y_test, y_proba),
            'threshold': threshold
        }
        
        # Additional metrics
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        metrics.update({
            'specificity': tn / (tn + fp),
            'g_mean': np.sqrt(metrics['recall'] * (tn / (tn + fp))),
            'pr_auc': auc(*precision_recall_curve(y_test, y_proba)[1::-1])
        })
        
        results.append(metrics)
        
    except Exception as e:
        print(f"Prediction error in iteration {i}: {e}")
        continue

# Convert to DataFrame
df_results = pd.DataFrame(results)

# Save results
df_results.to_csv('LOANDEFAULT_RF_Adaptive_Weighting(GSMOTE_NC).csv', index=False)

# Display results and feature importances
print(df_results.head())

# Feature importance analysis
feature_importances = pd.DataFrame({
    'feature': range(X_g_smote_imputed.shape[1]),
    'importance': weights
}).sort_values('importance', ascending=False)

print("\nTop 10 most important features:")
print(feature_importances.head(10))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score,
    confusion_matrix, precision_recall_curve, auc
)
from sklearn.utils import shuffle, resample
from sklearn.impute import SimpleImputer
from tqdm import tqdm

# Define adaptive weighting function
gprime = lambda w: 1. / (2. * np.sqrt(np.abs(w)) + np.finfo(float).eps)

# Initialize the imputer
imputer = SimpleImputer(strategy='mean')

# Apply imputer to data
X_S_NC_imputed = imputer.fit_transform(X_S_NC)
X_test_sc_imputed = imputer.transform(X_test_sc)

# Prepare results storage
results = []

for i in tqdm(range(1, 101), desc="Training Adaptive Random Forest"):
    # Random subsampling (80% of data)
    X_train_subset, y_train_subset = resample(
        X_S_NC_imputed, y_S_NC,
        n_samples=int(0.8 * len(X_S_NC_imputed)),
        random_state=i
    )
    
    # Shuffle data
    X_train_shuffled, y_train_shuffled = shuffle(X_train_subset, y_train_subset, random_state=i)
    X_train_array = np.asarray(X_train_shuffled)

    # Initialize weights
    weights = np.ones(X_train_array.shape[1])
    
    # Adaptive weighting iterations
    for _ in range(5):
        # Apply weights to features
        X_weighted = X_train_array / weights
        X_weighted = X_weighted + np.random.normal(0, 1e-3, X_weighted.shape)  # Small jitter
        
        try:
            model = RandomForestClassifier(
                n_estimators=250,
                max_depth=None,
                min_samples_split=2,
                min_samples_leaf=1,
                random_state=i,
                class_weight='balanced'
            )
            model.fit(X_weighted, y_train_shuffled)
            
            # Update weights using feature importances
            weights = gprime(model.feature_importances_)
            
        except Exception as e:
            print(f"Training error in iteration {i}: {e}")
            continue

    # Make predictions
    try:
        # Apply final weights to test data
        X_test_weighted = X_test_sc_imputed / weights
        
        y_proba = model.predict_proba(X_test_weighted)[:, 1]
        threshold = np.random.uniform(0.4, 0.6)
        y_pred = (y_proba >= threshold).astype(int)

        # Compute metrics
        metrics = {
            'iteration': i,
            'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred),
            'roc_auc': roc_auc_score(y_test, y_proba),
            'threshold': threshold
        }
        
        # Additional metrics
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        metrics.update({
            'specificity': tn / (tn + fp),
            'g_mean': np.sqrt(metrics['recall'] * (tn / (tn + fp))),
            'pr_auc': auc(*precision_recall_curve(y_test, y_proba)[1::-1])
        })
        
        results.append(metrics)
        
    except Exception as e:
        print(f"Prediction error in iteration {i}: {e}")
        continue

# Convert to DataFrame
df_results = pd.DataFrame(results)

# Save results
df_results.to_csv('LOANDEFAULT_RF_Adaptive_Weighting(SMOTE_NC).csv', index=False)

# Display results and feature importances
print(df_results.head())

# Feature importance analysis
feature_importances = pd.DataFrame({
    'feature': range(X_S_NC_imputed.shape[1]),
    'importance': weights
}).sort_values('importance', ascending=False)

print("\nTop 10 most important features:")
print(feature_importances.head(10))